<a href="https://colab.research.google.com/github/cminy/FC2022code/blob/2204_kyo/sql_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
g# mysql 사용을 위한 패키지 설치
!sudo apt-get install libmysqlclient-dev
!pip install mysqlclient

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmysqlclient-dev is already the newest version (5.7.37-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
# 패키지 코드를 사용할 수 있도록 패키지 import
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import MySQLdb

In [ ]:
db = MySQLdb.connect(host="0.tcp.ngrok.io", port=15181, user="root", passwd="mk246900", auth_plugin='mysql_native_password', db="world", charset="utf8")
curs = db.cursor()

In [ ]:
# 1. SQL 코드 실행 1 : 출력되는 데이터가 있을때 (데이터 출력)
QUERY = "SELECT * FROM country"
df = pd.read_sql(QUERY, db)
df.head(1)

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.0,NaN,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129.0,AW


In [ ]:
# 2. SQL 코드 실행 2 : 출력되는 데이터가 없을때 (설정)
QUERY = "USE sakila" # sakila 데이터 베이스로 이동
curs.execute(QUERY)

0

In [ ]:
# 현재 사용중인 데이터 베이스 확인
QUERY = "SELECT database()"
pd.read_sql(QUERY, db)

,database()
0,sakila


In [ ]:
QUERY = "USE world" # sakila 데이터 베이스로 이동
curs.execute(QUERY)

0

In [ ]:
QUERY = "SHOW tables;"
pd.read_sql(QUERY, db)

,Tables_in_world
0,addr
1,city
2,country
3,countrylanguage
4,user


In [ ]:
# qr = "drop view city900, city_900, cityy900;"
# curs.execute(qr)

In [ ]:
QUERY = "SELECT Code, Name, Population FROM country"
df = pd.read_sql(QUERY, db)
df.head(3)

,Code,Name,Population
0,ABW,Aruba,103000
1,AFG,Afghanistan,22720000
2,AGO,Angola,12878000


In [ ]:
# * : 모든의 의미를 가짐
# QUERY = "SELECT * FROM country"
# MySQL은 대소문자를 구분하지 않음(X)
QUERY = "SELECT Code, Name, Population FROM country"
df = pd.read_sql(QUERY, db)
df.tail(1)

,Code,Name,Population
238,ZWE,Zimbabwe,11669000


In [ ]:
# 산술 연산자
# 국가코드, 국가이름, 인구밀도(인구수/면적) 출력
QUERY = """
    SELECT code, name, population / surfacearea AS pps
    FROM country
"""
df = pd.read_sql(QUERY, db)
df.head(3)

,code,name,pps
0,ABW,Aruba,533.6788
1,AFG,Afghanistan,34.8418
2,AGO,Angola,10.3297


In [ ]:
# 국가코드, 국가이름, 1인당 GNP 출력 
qr = '''
  SELECT code, name, gnp / population AS gpp
  FROM country
'''
df = pd.read_sql(QUERY, db)
df.head(3)

,code,name,pps
0,ABW,Aruba,533.6788
1,AFG,Afghanistan,34.8418
2,AGO,Angola,10.3297


In [ ]:
# Asia 대륙에 속한 국가는 True(1)로 출력되고 아니면 False(0)으로 출력되는 컬럼을 추가
# 출력컬럼 : code, name, continent, is_asia
qr = '''
  SELECT code, name, continent, continent = "asia" AS is_asia
  FROM country
'''
df = pd.read_sql(qr, db)
df.head(3)

,code,name,continent,is_asia
0,ABW,Aruba,North America,0
1,AFG,Afghanistan,Asia,1
2,AGO,Angola,Africa,0


In [ ]:
# 인구수가 천만명 이상인 국가는 True(1) dkslaus False(0)를 출력
# 출력컬럼 : code, name, population, upper_1000
qr = '''
  SELECT code, name, population, population >= 1000*10000 AS upper_1000
  FROM country
'''
df = pd.read_sql(qr, db)
df.head(3)

,code,name,population,upper_1000
0,ABW,Aruba,103000,0
1,AFG,Afghanistan,22720000,1
2,AGO,Angola,12878000,1


In [ ]:
# 논리연산자 : 논리값을 연산
# OR : 둘다 False 일때 False
# AND : 둘다 True 일때 True

# 인구수 천만명 이상 이거나 GNP가 3천 이상인 국가를 1 출력
# 컬럼 : code, name, population, gnp, con1
QUERY = """
    SELECT code, name, population, gnp, (population >= 1000*10000) OR (gnp >= 3000) AS con1
    FROM country
"""
df = pd.read_sql(QUERY, db)
df.head()

,code,name,population,gnp,con1
0,ABW,Aruba,103000,828.0,0
1,AFG,Afghanistan,22720000,5976.0,1
2,AGO,Angola,12878000,6648.0,1
3,AIA,Anguilla,8000,63.2,0
4,ALB,Albania,3401200,3205.0,1


In [ ]:
# WHERE : 특정 조건으로 데이터를 필터링하고 출력하기
# 인구수가 1억 이상인 국가의 국가코드, 국가이름, 인구수 출력
QUERY = """
    SELECT code, name, population
    FROM country
    where population >= 10000*10000
"""
df = pd.read_sql(QUERY, db)
df.head()

,code,name,population
0,BGD,Bangladesh,129155000
1,BRA,Brazil,170115000
2,CHN,China,1277558000
3,IDN,Indonesia,212107000
4,IND,India,1013662000


In [ ]:
# 인구수가 8천만 ~ 1억 사이인 국가의 국가코드, 국가이름, 인구수 출력
QUERY = """
    SELECT code, name, population
    FROM country
    where (population >= 8000*10000) AND (population <=10000*10000)
"""
df = pd.read_sql(QUERY, db)
df.head()

,code,name,population
0,DEU,Germany,82164700
1,MEX,Mexico,98881000


In [ ]:
# BETWEEN
QUERY = """
    SELECT code, name, population
    FROM country
    WHERE population BETWEEN 8000 * 10000 AND 10000 * 10000
"""
df = pd.read_sql(QUERY, db)
df.head()

,code,name,population
0,DEU,Germany,82164700
1,MEX,Mexico,98881000


In [ ]:
# asia, africa 대륙에 포함되는 국가 출력
# 국가코드, 국가이름, 대륙
QUERY = """
    SELECT code, name, continent
    FROM country
    WHERE (continent = 'asia') OR (continent = 'africa')
"""
df = pd.read_sql(QUERY, db)
df.head()

,code,name,continent
0,AFG,Afghanistan,Asia
1,AGO,Angola,Africa
2,ARE,United Arab Emirates,Asia
3,ARM,Armenia,Asia
4,AZE,Azerbaijan,Asia


In [ ]:
# asia, africa 대륙에 포함되는 국가 출력
# 국가코드, 국가이름, 대륙
QUERY = """
    SELECT code, name, continent
    FROM country
    WHERE continent in ('asia','africa')
"""
df = pd.read_sql(QUERY, db)
df.head()

,code,name,continent
0,AFG,Afghanistan,Asia
1,AGO,Angola,Africa
2,ARE,United Arab Emirates,Asia
3,ARM,Armenia,Asia
4,AZE,Azerbaijan,Asia


In [ ]:
# asia, africa 대륙에 포함되는 국가 출력
# 국가코드, 국가이름, 대륙
QUERY = """
    SELECT code, name, continent
    FROM country
    WHERE continent not in ('asia','africa')
"""
df = pd.read_sql(QUERY, db)
df.head()

,code,name,continent
0,ABW,Aruba,North America
1,AIA,Anguilla,North America
2,ALB,Albania,Europe
3,AND,Andorra,Europe
4,ANT,Netherlands Antilles,North America


In [ ]:
# LIKE
# 국가코드, 국가이름, 대륙
# 국가코드 K로 시작
QUERY = """
    SELECT code, name, continent
    FROM country
    WHERE code LIKE "K%"
"""
df = pd.read_sql(QUERY, db)
df.head()

,code,name,continent
0,KAZ,Kazakstan,Asia
1,KEN,Kenya,Africa
2,KGZ,Kyrgyzstan,Asia
3,KHM,Cambodia,Asia
4,KIR,Kiribati,Oceania


In [ ]:
qr = "use sakila;"
curs.execute(qr)

0

In [ ]:
# film 테이블에서 영화설명에 robot이 들어간 영화 출력
# film_id, title, description, length
# 러닝타임 60분 ~ 100분 사이
QUERY = """
    SELECT film_id, title, description, length
    FROM film
    WHERE description LIKE "%robot%" AND (length between 60 and 100)
"""
df = pd.read_sql(QUERY, db)
df.head()

,film_id,title,description,length
0,25,ANGELS LIFE,A Thoughtful Display of a Woman And a Astronau...,74
1,63,BEDAZZLED MARRIED,A Astounding Character Study of a Madman And a...,73
2,85,BONNIE HOLOCAUST,A Fast-Paced Story of a Crocodile And a Robot ...,63
3,113,CALIFORNIA BIRDS,A Thrilling Yarn of a Database Administrator A...,75
4,138,CHARIOTS CONSPIRACY,A Unbelieveable Epistle of a Robot And a Husba...,71


In [ ]:
# # 코딩 잘하는척하기 hacker typer (C언어)

In [ ]:
qr = "use world;"
curs.execute(qr)

0

In [ ]:
# 정렬 ORDER BY
# 인구수 내림차순으로 국가코드, 국가이름, 인구수 출력
QUERY = """
    SELECT code, name, population
    FROM country
    ORDER BY population desc;
"""
df = pd.read_sql(QUERY, db)
df.head(10)

,code,name,population
0,CHN,China,1277558000
1,IND,India,1013662000
2,USA,United States,278357000
3,IDN,Indonesia,212107000
4,BRA,Brazil,170115000
5,PAK,Pakistan,156483000
6,RUS,Russian Federation,146934000
7,BGD,Bangladesh,129155000
8,JPN,Japan,126714000
9,NGA,Nigeria,111506000


In [ ]:
# 도시인구수 내림차순으로 정렬
# 국가코드, 도시이름, 도시인구수
QUERY = """
    SELECT countrycode, name, population
    FROM city
    ORDER BY population desc;
"""
df = pd.read_sql(QUERY, db)
df.head()

,countrycode,name,population
0,IND,Mumbai (Bombay),10500000
1,KOR,Seoul,9981619
2,BRA,São Paulo,9968485
3,CHN,Shanghai,9696300
4,IDN,Jakarta,9604900


In [ ]:
# 국가코드 내림차순, 도시인구수 오름차순으로 정렬
# 국가코드, 도시이름, 도시인구수
QUERY = """
    SELECT countrycode, name, population
    FROM city
    ORDER BY countrycode desc, population;
"""
df = pd.read_sql(QUERY, db)
df.head()

,countrycode,name,population
0,ZWE,Gweru,128037
1,ZWE,Mutare,131367
2,ZWE,Mount Darwin,164362
3,ZWE,Chitungwiza,274912
4,ZWE,Bulawayo,621742


In [ ]:
# LIMIT : 조회되는 데이터의 수를 제한
# 인구수 많은 도시 3개 출력
QUERY = """
    SELECT countrycode, name, population
    FROM city
    ORDER BY population DESC
    limit 3
"""
df = pd.read_sql(QUERY, db)
df.head()

,countrycode,name,population
0,IND,Mumbai (Bombay),10500000
1,KOR,Seoul,9981619
2,BRA,São Paulo,9968485


In [ ]:
# 4~5위 출력
QUERY = """
    SELECT countrycode, name, population
    FROM city
    ORDER BY population DESC
    limit 3, 2
"""
df = pd.read_sql(QUERY, db)
df.head()

,countrycode,name,population
0,CHN,Shanghai,9696300
1,IDN,Jakarta,9604900


In [ ]:
# DML : DATA : CRUD : create, read, update, delete
# DDL : DATABASE, TABLE : CRUD : create, read, update, delete

In [ ]:
QUERY = """
    SELECT countrycode, name, population
    FROM city
    ORDER BY population DESC
    LIMIT 20
"""
df = pd.read_sql(QUERY, db)
df.head()

,countrycode,name,population
0,IND,Mumbai (Bombay),10500000
1,KOR,Seoul,9981619
2,BRA,São Paulo,9968485
3,CHN,Shanghai,9696300
4,IDN,Jakarta,9604900


In [ ]:
df.to_csv('datatest.csv', index=False, encoding = "utf-8-sig")

In [ ]:
df2 = pd.read_csv("datatest.csv")
df2

,countrycode,name,population
0,IND,Mumbai (Bombay),10500000
1,KOR,Seoul,9981619
2,BRA,São Paulo,9968485
3,CHN,Shanghai,9696300
4,IDN,Jakarta,9604900
5,PAK,Karachi,9269265
6,TUR,Istanbul,8787958
7,MEX,Ciudad de México,8591309
8,RUS,Moscow,8389200
9,USA,New York,8008278


In [ ]:
# Quiz : 한국 도시 중에서 인구수가 100만이 넘는 도시를 인구수 순으로 내림차순 출력
# 컬럼 : 
qr = '''
select * 
from city
where (population >= 100*10000) AND countrycode = "KOR"
order by population desc
limit 3
'''
an = pd.read_sql(qr,db)
an.head(3)

,ID,Name,CountryCode,District,Population
0,2331,Seoul,KOR,Seoul,9981619
1,2332,Pusan,KOR,Pusan,3804522
2,2333,Inchon,KOR,Inchon,2559424


In [ ]:
# 데이터 타입
# 숫자 : 정수int, 실수 float
# 문자열 : char, varchar, text
# 날짜시간 : datetime, timestamp